In [1]:
import os
from glob import glob
from tqdm import tqdm
from pathlib import Path
import time

import pandas as pd
import numpy as np
from src.stats import calculate_p

/scratch4/lisik3/emcmaho7/SIfMRI_modeling/notebooks


In [ ]:
class ModelAveraging:
    def __init__(self, args):
        self.process = 'ModelAveraging'
        self.model_class = 'VisionNeuralEncoding'
        self.model_subpath = 'grouped_average'
        self.voxel_id = 9539 #test voxel in EVC
        self.top_dir = f'/home/emcmaho7/scratch4-lisik3/emcmaho7/SIfMRI_modeling/data/interim'

        self.ut_path = f'{top_dir}/{self.process}'
        Path(out_path).mkdir(exist_ok=True, parents=True)
        cols2keep = ['voxel_id', 'layer_relative_depth',
                    'train_score', 'test_score',
                    'r_null_dist', 'r_var_dist']

    def run(self): 
        if self.model_subpath is not None: 
            file_path = f'{self.top_dir}/{self.model_class}/{self.model_subpath}/'
        else: 
            file_path = f'{self.top_dir}/{self.model_class}'
    
        files = glob(f'{file_path}/*.pkl.gz')
        n_files = len(files)
        print(f'{len(files)} files found')

        df, n_files = load_files(files)
        print(df.loc[df.voxel_id == voxel_id])

        df = divide_pd_array(df, n_files)
        print(df.loc[df.voxel_id == voxel_id])

        # calculate the confidence interval
        df[['lower_ci', 'upper_ci']] = df['r_var_dist'].apply(lambda arr: pd.Series(compute_confidence_intervals(arr)))
        print(df.loc[df.voxel_id == voxel_id])

        # calculate the p value
        df['p'] = df.apply(calculate_p_df, axis=1)
        print(df.loc[df.voxel_id == voxel_id])

        save_start = time.time()
        df.to_pickle(f'{out_path}/{model_class}_{model_subpath}.pkl.gz')
        save_time = time.time() - save_start
        elapsed = time.strftime("%H:%M:%S", time.gmtime(save_time))
        print(f'Saved in {elapsed}!')

342 files found


In [6]:
def sum_of_arrays(series):
    # Stack arrays vertically and compute sum along the first axis (rows)
    return np.nansum(np.vstack(series), axis=0)

def compute_confidence_intervals(arr):
    lower = np.nanpercentile(arr, 2.5)
    upper = np.nanpercentile(arr, 97.5)
    return lower, upper

def calculate_p_df(row):
    r_value = row['test_score']  # The 'r' value for the current row
    r_null_array = row['r_null_dist']  # The 'r_null' array for the current row
    return calculate_p(r_null_array, r_value, n_perm_=len(r_null_array), H0_='greater')

def load_files(files):
    # Load the files and sum them up 
    df = None
    n_final_files = 0
    for file in tqdm(files, total=n_files, desc='Loading files'): 
        try: 
            pkl = pd.read_pickle(file)[cols2keep]

            # remove voxels not in roi
            if 'roi_name' in pkl.columns: 
                pkl = pkl.loc[pkl.roi_name != 'none'].reset_index()

            if df is None: 
                df = pkl
            else:
                #After the first file has been loaded, concatenate the data and add it together
                df = pd.concat([df, pkl])
                df = df.groupby('voxel_id').agg({
                                                'train_score': 'sum',
                                                'test_score': 'sum',
                                                'layer_relative_depth': 'sum',
                                                'r_null_dist': sum_of_arrays,
                                                'r_var_dist': sum_of_arrays
                                                }).reset_index()
            n_final_files += 1
        except:
            print(f'could not load {file}')
    return df 

def divide_pd_array(df, n): 
    def divide_array(arr):
        return arr / n

    # Get the mean by averaging by the total number of models
    columns_to_divide = ['train_score', 'test_score', 'layer_relative_depth', 'r_null_dist', 'r_var_dist']
    for col in columns_to_divide:
        if isinstance(df[col][0], np.ndarray):
            # Apply division to arrays
            df[col] = df[col].apply(divide_array)
        else:
            # Apply division to numeric columns
            df[col] = df[col] / n
    print(df.loc[df.voxel_id == voxel_id])
    return df 